### Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from scipy.fftpack import fft
from scipy.stats import norm
import pywt
import aeon
from aeon.classification.distance_based import KNeighborsTimeSeriesClassifier
import math
from sklearn import svm, neighbors
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
#from sklearn.model_selection import LeaveOneOut
from tqdm import tqdm
from sklearn.metrics import accuracy_score

#grindSearch
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


### Dados

In [ ]:
try:
    train_data = pd.read_parquet('\DS\Parquet_Quali\CBF_TRAIN.parquet')
    test_data = pd.read_parquet('\DS\Parquet_Quali\CBF_TEST.parquet')
except FileNotFoundError:
    print("Ensure the Parquet files are in the correct path.")
    raise

In [ ]:
X_train = train_data.drop('target', axis=1).values
y_train = train_data['target'].values

X_test = test_data.drop('target', axis=1).values
y_test = test_data['target'].values

### Função de transformação dos dados.

In [ ]:
def sax_transform(series, w, a):
    paa = [series[i:i + w].mean() for i in range(0, len(series), w)]
    
    if np.std(paa) != 0:
        paa = (paa - np.mean(paa)) / np.std(paa)
    else:
        paa = paa - np.mean(paa)
    
    breakpoints = norm.ppf(np.linspace(0, 1, a+1)[1:-1])
    sax_symbols = np.array(range(a))
    sax_representation = sax_symbols[np.digitize(paa, breakpoints)]
    
    return sax_representation

def transform_data(X, num_features=5):
    a = 5
    w = int(X.shape[1] / num_features)  # Ajuste do tamanho da janela baseado no número de características desejado
    
    X_sax = np.array([sax_transform(row, w, a) for row in X])
    X_fft = np.abs(fft(X, axis=1))
    
    coeffs_cA, coeffs_cD = pywt.dwt(X, 'db4', axis=1)
    X_dwt = np.hstack((coeffs_cA, coeffs_cD))
    
    X_paa = np.column_stack([X[:, i:i+2].mean(axis=1) for i in range(0, X.shape[1], 2)])

    return {
        "TS": X,
        "FFT": X_fft,
        "DWT": X_dwt,
        "PAA": X_paa,
        "SAX": X_sax
    }

In [ ]:
from joblib import Parallel, delayed
# Carregando os dados
train, val, label, true_label = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



params = {knn, nb, rf, gbc, svc}
# Definindo a grade de hiperparâmetros para cada classificador
param_grids = {
    'KNN': {
        'knn__n_neighbors': [1, 3, 5, 7],
        'knn__weights': ['distance'],
        'knn__distance': ['dtw', 'ddtw','wdtw','wddtw', 'lcss', 'erp', 'msm']
    },
    'GaussianNB': {'var_smoothing': np.logspace(0,-9, num=100)
    },
    'RandomForest': {
        'rf__n_estimators': [50, 100, 200],
        'rf__max_depth': [None, 10, 20]
    },
    'GradientBoosting': {
        'gb__n_estimators': [50, 100, 200],
        'gb__learning_rate': [0.01, 0.1, 0.5]
    },
    'SVC': {
        'svc__C': [0.1, 1, 10, 100, 1000],
        'svc__kernel': ['linear', 'rbf', 'poly'],
        'svc__gamma': [1, 0.1, 0.01, 0.001, 0.0001]
    }
}

# Função para executar a busca em grade para um classificador específico
def run_grid_search(clf_name):
    pipeline = classifiers[clf_name]
    param_grid = param_grids[clf_name]
    grid_search = GridSearchCV(pipeline, param_grid, cv=3)
    grid_search.fit(train, label)
    accuracy = grid_search.best_estimator_.score(val, true_label)
    return clf_name, grid_search.best_estimator_, grid_search.best_params_, accuracy

# Executando a busca em grade para cada classificador em paralelo
results = Parallel(n_jobs=-1)(
    delayed(run_grid_search)(clf_name) for clf_name in classifiers.keys()
)

# Imprimindo os resultados
for clf_name, best_model, best_params, accuracy in results:
    print(clf_name)
    print("Accuracy:", accuracy)
    print("Best parameters:", best_params)
    print()


### Seleção do modelo extrator e modelo classificador

In [ ]:
def select_model(option, random_state):
    if option == '1nn':
        return neighbors.KNeighborsClassifier(metric='euclidean', n_neighbors=1)
    elif option == '3nn':
        return KNeighborsTimeSeriesClassifier(distance='dtw', n_neighbors=3)
    elif option == 'svm':
        return svm.SVC(C=10, gamma=0.01, kernel='sigmoid', probability=True)
    elif option == 'gbc':
        return GradientBoostingClassifier(n_estimators=100, random_state=random_state)
    elif option == 'nb':
        return GaussianNB()
    elif option == 'xgb':
        return xgb.XGBClassifier(random_state=random_state)
    elif option == 'rd':
        return RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
    else:
        return RandomForestClassifier(n_estimators=100,random_state=random_state)

### Treino do modelos extrator e classificador

In [ ]:
def train_with_meta_classifier(X_train, y_train, base_option='random_forest', meta_option='1nn', random_state=42, cv=3):
    trained_models = {}  # Salvar modelos treinados para cada transformação
    
    X_train_transformed = transform_data(X_train)  # Transformar todo o conjunto de treino

    # Treinar um modelo para cada transformação e salvar no dicionário
    for rep, X_trans in tqdm(X_train_transformed.items(), ascii=True, desc="Training Base Models"):
        model = select_model(base_option, random_state)
        scores = cross_val_score(model, X_trans, y_train, cv=cv)  # Calcula a acurácia em k-fold cross-validation
        accuracy = np.mean(scores) if len(scores) > 0 else 0.0  # Calcula a média das acurácias
        model._accuracy = accuracy  # Adiciona um atributo _accuracy ao modelo
        model.fit(X_trans, y_train)
        trained_models[rep] = model
        
    # Preparar dados para o meta-classificador
    meta_features = []
    weights = {}  # Dicionário para armazenar os pesos dos modelos base
    for rep, model in trained_models.items():
        weight = accuracy_score(y_train, model.predict(X_train_transformed[rep])) ** 4  # Calcula o peso usando a acurácia elevada à quarta potência
        weights[rep] = weight
    
    for i in range(X_train.shape[0]):
        instance_sums = np.zeros((1, len(np.unique(y_train))))  # Inicializar a soma das probabilidades para esta instância
        for rep, model in trained_models.items():
            proba = model.predict_proba(X_train_transformed[rep][i].reshape(1, -1))
            # Multiplicar as probabilidades previstas pelo peso do modelo
            instance_sums += proba * weights[rep]
        # Normalizar as somas pelo somatório dos pesos
        instance_sums /= np.sum(list(weights.values()))
        meta_features.append(instance_sums.flatten())  # Estender a lista com as probabilidades normalizadas

    meta_features = np.array(meta_features)
    np.savetxt("meta-features-train.csv", meta_features, delimiter=",")
    
    # Treinar o meta-classificador
    meta_classifier = select_model(meta_option, random_state)
    meta_classifier.fit(meta_features, y_train)
    
    return trained_models, meta_classifier


### Predicao do meta-classificador

In [ ]:
def predict_with_meta_classifier(X_test, trained_base_models, trained_meta_classifier, weights=None):
    predictions = []
    meta_features_test = []  # Inicialize uma lista para armazenar todos os meta-recursos dos dados de teste
    
    for i in tqdm(range(len(X_test)), ascii=True, desc="Testing Instances"):
        x_instance = X_test[i].reshape(1, -1)
        x_transformed = transform_data(x_instance)
        
        instance_sums = np.zeros((1, len(np.unique(y_train))))  # Inicializar a soma das probabilidades para esta instância
        for rep, model in trained_base_models.items():
            proba = model.predict_proba(x_transformed[rep][0].reshape(1, -1))  # Ajuste aqui para pegar o primeiro elemento
            # Adicionar o peso da probabilidade prevista para cada classe à soma correspondente
            instance_sums += proba * (weights[rep] if weights else 1)
        # Normalizar as somas pelo somatório dos pesos
        instance_sums /= np.sum(weights) if weights else len(trained_base_models)
        
        meta_feature = instance_sums.flatten().reshape(1, -1)
        predictions.append(trained_meta_classifier.predict(meta_feature)[0])  # Adicionar a previsão à lista de previsões
        
        meta_features_test.append(meta_feature.flatten())  # Adicionar meta-recursos da instância atual à lista
    
    # Converter a lista de meta-recursos dos dados de teste em um array numpy
    meta_features_test = np.array(meta_features_test)

    # Salvar todos os meta-recursos dos dados de teste em um arquivo CSV
    np.savetxt("meta-features-test.csv", meta_features_test, delimiter=",")
    
    return predictions


### Testando um único modelo - Random Forest como extrator e SVM como meta-classificador

In [ ]:
# Treino
trained_base_models, meta_classifier = train_with_meta_classifier(X_train, y_train, base_option='random_forest', meta_option='svm', random_state=42)

# Teste
predictions_test_meta = predict_with_meta_classifier(X_test, trained_base_models, meta_classifier)

# Resultado
test_accuracy_meta = np.mean(predictions_test_meta == y_test)

print(f'Accuracy: {test_accuracy_meta}')

### Testando RidgeClassifier

In [ ]:
# Treino
trained_base_models, meta_classifier = train_with_meta_classifier(X_train, y_train, base_option='random_forest', meta_option='rd', random_state=42)

# Teste
predictions_test_meta = predict_with_meta_classifier(X_test, trained_base_models, meta_classifier)

# Resultado
test_accuracy_meta = np.mean(predictions_test_meta == y_test)

print(f'Accuracy: {test_accuracy_meta}')

### Testando um único modelo - SVM como extrator e meta-classificador

In [ ]:
# Treino
trained_base_models, meta_classifier = train_with_meta_classifier(X_train, y_train, base_option='svm', meta_option='rd', random_state=42)

# Teste
predictions_test_meta = predict_with_meta_classifier(X_test, trained_base_models, meta_classifier)

# Resultado
test_accuracy_meta = np.mean(predictions_test_meta == y_test)
print(f'Accuracy: {test_accuracy_meta}')

### Treino em loop de todas as opções de classificadores disponiveis no Select Model.

In [ ]:
algos = ['1nn', '3nn', 'svm', 'nb', 'gbc']
for algo in algos:
    
    print(f'Meta-classificador com modelo extrator {algo.upper()}')
    
    # Training
    try:
        trained_base_models, meta_classifier = train_with_meta_classifier(X_train, y_train, base_option=algo, meta_option=algo)
        # Testing
        predictions_test_meta = predict_with_meta_classifier(X_test, trained_base_models, meta_classifier)
    
        test_accuracy_meta = np.mean(predictions_test_meta == y_test)
        print(f'Acurácia do teste usando o meta-classificador com modelo extrator {algo}: {test_accuracy_meta}')
    except Exception as e:
        print(f"Ocorreu um erro no teste com o {algo}: {e}")
    print("-------------------------------")
print('RF (10x)')
for i in range(1):
    print(f'RF: Random seed = {i}')
    try:
        trained_base_models, meta_classifier = train_with_meta_classifier(X_train, y_train, base_option='random_forest', meta_option='random_forest', random_state=i)
    
        # Testing
        predictions_test_meta = predict_with_meta_classifier(X_test, trained_base_models, meta_classifier)
    
        test_accuracy = np.mean(predictions_test_meta == y_test)
        print(f'Acurácia do teste usando seed {i}: {test_accuracy}')
    except Exception as e:
        print(f"Ocorreu um erro no teste com o RF (seed {i}): {e}")
    print("-------------------------------")

### Utilizando um classificador raso com os dados extraidos do meta-classificador contra os dados puros

In [ ]:
#Teste utilizando o classificador SVM
meta_attrib_train = np.loadtxt("meta-features-train.csv", delimiter=",")
meta_attrib_test = np.loadtxt("meta-features-test.csv", delimiter=",")

clf = svm.SVC(probability=True)
clf.fit(meta_attrib_train, y_train)
y_hat = clf.predict(meta_attrib_test)
test_accuracy_meta = np.mean(y_hat == y_test)
print(f"accuracy: {test_accuracy_meta}")

In [ ]:
clf_2 = svm.SVC(probability=True)
clf_2.fit(X_train, y_train)
y_hat_ = clf_2.predict(X_test)
test_accuracy_meta_2 = np.mean(y_hat_ == y_test)
print(f"accuracy: {test_accuracy_meta_2}")

### Gráfico das diferenças de dados

In [ ]:
import matplotlib.pyplot as plt

y1 = y_hat  # depois da transformação
y2 = y_test  

z1 = y_hat_ #antes da transformação
z2 = y_test

#suavizar os dados do gráfico
window_size = 50
y1_smoothed = pd.Series(y1).rolling(window=window_size).mean()
y2_smoothed = pd.Series(y2).rolling(window=window_size).mean()
z1_smoothed = pd.Series(z1).rolling(window=window_size).mean()
z2_smoothed = pd.Series(z2).rolling(window=window_size).mean()

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5), layout='constrained')

# Conjunto de validação do classificador
axs[0].set_title('Antes da transformação')
axs[0].plot(z1_smoothed, label='Treino')
axs[0].plot(z2_smoothed, label='Teste')
axs[0].set_xlabel('Tempo (s)')
axs[0].set_ylabel('Treino')
axs[0].grid(True)

# Conjunto de validação do meta-classificador
axs[1].set_title('Depois da transformação')
axs[1].plot(y1_smoothed, label='Treino')
axs[1].plot(y2_smoothed, label='Teste')
axs[1].set_xlabel('Tempo (s)')
axs[1].set_ylabel('Treino')
axs[1].grid(True)

plt.legend()
plt.show()



In [ ]:
w1 = y_hat  # meta-classificador
w2 = y_hat_ #classificação

# Suavizar os dados do gráfico
window_size = 90
w1_smoothed = pd.Series(w1).rolling(window=window_size).mean()
w2_smoothed = pd.Series(w2).rolling(window=window_size).mean()

# Plotar os dados
plt.figure(figsize=(10, 6))
plt.plot(w1_smoothed, label='w1 (Classificação usando meta-caracteristicas)')
plt.plot(w2_smoothed, label='w2 (classificação utilizando dados brutos)')
plt.xlabel('Tempo (s)')
plt.ylabel('Valores suavizados')
plt.title('Comparação entre os resultados de um SVM')
plt.legend()
plt.grid(True)
plt.show()
